# CME 241 (Winter 2021) -- Assignment 7

## Question 2

In [1]:
from __future__ import annotations

from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Sequence, Callable, Tuple, Iterator, TypeVar, Generic
from pprint import pprint

from operator import itemgetter
import numpy as np

from rl.distribution import Gaussian, Choose, SampledDistribution
from rl.markov_decision_process import MarkovDecisionProcess, Policy
from rl.function_approx import DNNSpec, AdamGradient, DNNApprox
from rl.approximate_dynamic_programming import back_opt_vf_and_policy
from rl.approximate_dynamic_programming import back_opt_qvf

If we relax the requirement that `risky_return_distributions` can be any generic distributions, and instead impose a certain type of distribution, then we can drastically speed up our program. This is due to the fact we can now directly compute the wealth distribution $W_{t+1}$ and thus performing each `step()` call will no longer need to generate thousands of samples. 

To do this, we introduce a custom distribution `WealthRewardDistribution` that computes expectations in $O(1)$.

In [2]:
T = Tuple[float, float]

class WealthRewardGaussian(SampledDistribution[T]):
    """TODO"""
        
    def __init__(self, mu: float, sigma: float, reward: float) -> None:
        self.mu: float = mu
        self.sigma: float = sigma
        self.reward: float = reward
            
        _sampler = lambda: (
            np.random.normal(loc=self.mu, scale=self.sigma), self.reward
        )
        super().__init__(
            sampler=_sampler,
            expectation_samples=1
        )

In [3]:
@dataclass(frozen=True)
class AssetAllocDiscrete:
    risky_return_distributions: Sequence[Gaussian[float]]
    riskless_returns: Sequence[float]
    utility_func: Callable[[float], float]
    risky_alloc_choices: Sequence[float]
    feature_functions: Sequence[Callable[[Tuple[float, float]], float]]
    dnn_spec: DNNSpec
    initial_wealth_distribution: Gaussian[float]

    def time_steps(self) -> int:
        return len(self.risky_return_distributions)

    def uniform_actions(self) -> Choose[float]:
        return Choose(set(self.risky_alloc_choices))

    def get_mdp(self, t: int) -> MarkovDecisionProcess[float, float]:
        """
        State is Wealth W_t, Action is investment in risky asset (= x_t)
        Investment in riskless asset is W_t - x_t
        """

        distr: Gaussian[float] = self.risky_return_distributions[t]
        rate: float = self.riskless_returns[t]
        alloc_choices: Sequence[float] = self.risky_alloc_choices
        steps: int = self.time_steps()
        utility_f: Callable[[float], float] = self.utility_func

        class AssetAllocMDP(MarkovDecisionProcess[float, float]):
            def step(self, wealth: float, alloc: float) -> WealthRewardGaussian:
                expected_wealth = alloc * distr.μ + (wealth - alloc) * (1 + rate) 
                reward: float = utility_f(expected_wealth) if t == steps - 1 else 0.0    
                wealth = WealthRewardGaussian(
                    mu=expected_wealth,
                    sigma=distr.σ,
                    reward=reward
                )
                return wealth

            def actions(self, wealth: float) -> Sequence[float]:
                return alloc_choices

        return AssetAllocMDP()

    def get_qvf_func_approx(self) -> DNNApprox[Tuple[float, float]]:

        adam_gradient: AdamGradient = AdamGradient(
            learning_rate=0.1, decay1=0.9, decay2=0.999
        )
        return DNNApprox.create(
            feature_functions=self.feature_functions,
            dnn_spec=self.dnn_spec,
            adam_gradient=adam_gradient,
        )

    def get_states_distribution(self, t: int) -> SampledDistribution[float]:

        actions_distr: Choose[float] = self.uniform_actions()

        def states_sampler_func() -> float:
            wealth: float = self.initial_wealth_distribution.sample()
            for i in range(t):
                distr: Distribution[float] = self.risky_return_distributions[i]
                rate: float = self.riskless_returns[i]
                alloc: float = actions_distr.sample()
                wealth = alloc * (1 + distr.sample()) + (wealth - alloc) * (1 + rate)
            return wealth

        return SampledDistribution(states_sampler_func)

    def backward_induction_qvf(self) -> Iterator[DNNApprox[Tuple[float, float]]]:

        init_fa: DNNApprox[Tuple[float, float]] = self.get_qvf_func_approx()

        mdp_f0_mu_triples: Sequence[
            Tuple[
                MarkovDecisionProcess[float, float],
                DNNApprox[Tuple[float, float]],
                SampledDistribution[float],
            ]
        ] = [
            (self.get_mdp(i), init_fa, self.get_states_distribution(i))
            for i in range(self.time_steps())
        ]

        num_state_samples: int = 300
        error_tolerance: float = 1e-6

        return back_opt_qvf(
            mdp_f0_mu_triples=mdp_f0_mu_triples,
            γ=1.0,
            num_state_samples=num_state_samples,
            error_tolerance=error_tolerance,
        )

    def get_vf_func_approx(
        self, ff: Sequence[Callable[[float], float]]
    ) -> DNNApprox[float]:

        adam_gradient: AdamGradient = AdamGradient(
            learning_rate=0.1, decay1=0.9, decay2=0.999
        )
        return DNNApprox.create(
            feature_functions=ff, dnn_spec=self.dnn_spec, adam_gradient=adam_gradient
        )

    def backward_induction_vf_and_pi(
        self, ff: Sequence[Callable[[float], float]]
    ) -> Iterator[Tuple[DNNApprox[float], Policy[float, float]]]:

        init_fa: DNNApprox[float] = self.get_vf_func_approx(ff)

        mdp_f0_mu_triples: Sequence[
            Tuple[
                MarkovDecisionProcess[float, float],
                DNNApprox[float],
                SampledDistribution[float],
            ]
        ] = [
            (self.get_mdp(i), init_fa, self.get_states_distribution(i))
            for i in range(self.time_steps())
        ]

        num_state_samples: int = 300
        error_tolerance: float = 1e-8

        return back_opt_vf_and_policy(
            mdp_f0_mu_triples=mdp_f0_mu_triples,
            γ=1.0,
            num_state_samples=num_state_samples,
            error_tolerance=error_tolerance,
        )

In [4]:
steps: int = 4
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_var: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: -np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(
    2 / 3 * base_alloc, 4 / 3 * base_alloc, 11
)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = [
    lambda _: 1.0,
    lambda w_x: w_x[0],
    lambda w_x: w_x[1],
    lambda w_x: w_x[1] * w_x[1],
]
dnn: DNNSpec = DNNSpec(
    neurons=[],
    bias=False,
    hidden_activation=lambda x: x,
    hidden_activation_deriv=lambda y: np.ones_like(y),
    output_activation=lambda x: -np.sign(a) * np.exp(-x),
    output_activation_deriv=lambda y: -y,
)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_var)

aad: AssetAllocDiscrete = AssetAllocDiscrete(
    risky_return_distributions=risky_ret,
    riskless_returns=riskless_ret,
    utility_func=utility_function,
    risky_alloc_choices=alloc_choices,
    feature_functions=feature_funcs,
    dnn_spec=dnn,
    initial_wealth_distribution=init_wealth_distr,
)

In [5]:
it_qvf: Iterator[DNNApprox[Tuple[float, float]]] = aad.backward_induction_qvf()

print("Backward Induction on Q-Value Function")
print("--------------------------------------")
print()
for t, q in enumerate(it_qvf):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = max(
        ((q.evaluate([(init_wealth, ac)])[0], ac) for ac in alloc_choices),
        key=itemgetter(0),
    )[1]
    val: float = max(q.evaluate([(init_wealth, ac)])[0] for ac in alloc_choices)
    print(f"Opt Risky Allocation = {opt_alloc:.3f}, Opt Val = {val:.3f}")
    print("Optimal Weights below:")
    for wts in q.weights:
        pprint(wts.weights)
    print()

print("Analytical Solution")
print("-------------------")
print()

for t in range(steps):
    print(f"Time {t:d}")
    print()
    left: int = steps - t
    growth: float = (1 + r) ** (left - 1)
    alloc: float = base_alloc / growth
    val: float = (
        -np.exp(
            -excess * excess * left / (2 * var) - a * growth * (1 + r) * init_wealth
        )
        / a
    )
    bias_wt: float = excess * excess * (left - 1) / (2 * var) + np.log(np.abs(a))
    w_t_wt: float = a * growth * (1 + r)
    x_t_wt: float = a * excess * growth
    x_t2_wt: float = -var * (a * growth) ** 2 / 2

    print(f"Opt Risky Allocation = {alloc:.3f}, Opt Val = {val:.3f}")
    print(f"Bias Weight = {bias_wt:.3f}")
    print(f"W_t Weight = {w_t_wt:.3f}")
    print(f"x_t Weight = {x_t_wt:.3f}")
    print(f"x_t^2 Weight = {x_t2_wt:.3f}")
    print()

Backward Induction on Q-Value Function
--------------------------------------

Time 0

Opt Risky Allocation = 1.000, Opt Val = -19.233
Optimal Weights below:
array([[-3.200457  ,  1.33488506, -1.06969302, -0.02136637]])

Time 1

Opt Risky Allocation = 1.000, Opt Val = -6.337
Optimal Weights below:
array([[-1.85315566,  1.22131011, -1.283878  ,  0.06933296]])

Time 2

Opt Risky Allocation = 1.000, Opt Val = -2.269
Optimal Weights below:
array([[-0.94338294,  1.15439029, -1.04392853,  0.01341401]])

Time 3

Opt Risky Allocation = 1.000, Opt Val = -0.878
Optimal Weights below:
array([[-4.56861237e-04,  1.07000039e+00, -9.39387913e-01,
        -1.97487816e-04]])

Analytical Solution
-------------------

Time 0

Opt Risky Allocation = 1.224, Opt Val = -0.225
Bias Weight = 0.135
W_t Weight = 1.311
x_t Weight = 0.074
x_t^2 Weight = -0.030

Time 1

Opt Risky Allocation = 1.310, Opt Val = -0.257
Bias Weight = 0.090
W_t Weight = 1.225
x_t Weight = 0.069
x_t^2 Weight = -0.026

Time 2

Opt Risky A